In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
#loading the data generated in notMNIST.ipynb

pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
#Reformatting data into a shape that's more adapted to the models we're going to train:
#1. data as a flat matrix, and
#2. labels as float 1-hot encodings

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [6]:
#Introducing and tuning L2 regularization for both logistic and neural network models. L2 model amounts to addying a penalty on the norms of the weights of the loss. In tensorflow we can compute the L2 loss for a tensor t using nn.l2_loss(t). The right amount of rregularization should improve our validation/test accuracy.

#Multinomial logistic regression with L2 loss function. L' = L + B*(w1*w1 + w2*w2 + w3*w3 + .... + wn*wn)/2
#loading data and building computation graph.

#this is to expedite the process.
train_subset = 10000
#this is a good beta value to start with.
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    #input data
    #they are all constnts
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #training computation
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    #original loss function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits = logits))
    
    #loss function using L2 regularization
    regularizer = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(loss + beta * regularizer)
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    
    #predictions for training, validation and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
#running computation and iterating.

num_steps = 801

def accuracy(predictions, labels):
    return(100.0 * np.sum(np.argmax(predictions, 1)==np.argmax(labels, 1))/predictions.shape[0])

with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if(step%100==0):
            print("loss at step %d: %f" %(step, l))
            print("Training accuracy: %.1f%%" %accuracy(predictions, train_labels[:train_subset, :]))
            print("Validation accuracy: %.1f%%" %accuracy(valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" %accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
loss at step 0: 44.896332
Training accuracy: 8.4%
Validation accuracy: 9.5%
loss at step 100: 21.687206
Training accuracy: 66.7%
Validation accuracy: 65.6%
loss at step 200: 14.247317
Training accuracy: 72.6%
Validation accuracy: 70.8%
loss at step 300: 9.497972
Training accuracy: 75.3%
Validation accuracy: 73.5%
loss at step 400: 6.412051
Training accuracy: 77.4%
Validation accuracy: 75.3%
loss at step 500: 4.404790
Training accuracy: 79.2%
Validation accuracy: 76.8%
loss at step 600: 3.101700
Training accuracy: 80.6%
Validation accuracy: 78.1%
loss at step 700: 2.256930
Training accuracy: 82.0%
Validation accuracy: 79.1%
loss at step 800: 1.709228
Training accuracy: 82.9%
Validation accuracy: 79.9%
Test accuracy: 87.5%
